## Evaluation

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5" 

import random
import logging
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.io import loadmat, savemat

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torch.nn import L1Loss, CrossEntropyLoss
from torch.cuda.amp import GradScaler, autocast

import torchvision
import torchvision.transforms as T
import torchvision.models as models
from collections import Counter
from torchvision import models
import os
import sys
device = 'cuda' if torch.cuda.is_available() else 'cpu'

import torch.nn.functional as F
from mamba_ssm.ops.selective_scan_interface import selective_scan_fn
from einops import rearrange

/home/skumar4/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from mamba_vhs.dataset import DogHeartDataset, get_transform


resized_image_size = 512
true_batch_size = 256
accumulation_steps = 8
root_db_folder= "../data"

test_dataset = DogHeartDataset(f'{root_db_folder}/Test_Images', get_transform(resized_image_size))
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=8)


Number of samples in the test dataset: 4274


### Calculate Test Accuracy 

In [ ]:
def test_accuracy(model, test_loader, device, calc_vhs, get_labels, logger=None):
    model.eval()
    total_correct = 0
    total_samples = 0

    all_true_labels = []
    all_pred_labels = []

    progress = tqdm(test_loader, desc="Testing", leave=False, unit="batch")

    with torch.no_grad():
        for batch_idx, (idx, images, _, vhs) in enumerate(progress):
            images = images.to(device)
            vhs = vhs.to(device)

            outputs = model(images).squeeze()
            pred_vhs = calc_vhs(outputs).squeeze()

            pred_classes = get_labels(pred_vhs).cpu().numpy()
            true_classes = get_labels(vhs).cpu().numpy()

            total_correct += (pred_classes == true_classes).sum()
            total_samples += len(true_classes)

            all_true_labels.extend(true_classes)
            all_pred_labels.extend(pred_classes)

            progress.set_postfix({
                "Accuracy": f"{(total_correct / total_samples):.4f}"
            })

    progress.close()
    accuracy = total_correct / total_samples

    print(f"Test Accuracy: {accuracy:.4f}")
    return accuracy


In [ ]:
from mamba_vhs.mamba_vhs import get_mamba_vhs_model

checkpoint_path = '/scratch/genai/dog_heart/models/custom2/20250507_044926/iter_10/models/bm_15.pth'
model = get_mamba_vhs_model(checkpoint_path=checkpoint_path)
model = model.to(device)

#### Mamba Vision L

In [7]:
## Mamba Vision
import os
import torch
from mamba_vision import mamba_vision_L

# Define the pretrained model URL
pretrained_url = "https://huggingface.co/nvidia/MambaVision-L-1K/resolve/main/mambavision_large_1k.pth.tar"
pretrained_path = "/scratch/genai/vhs_model/mamba_vision_L/last_model.pth" #/tmp/mamba_vision_L.pth.tar"

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Download the pretrained weights if not already downloaded
if not os.path.exists(pretrained_path):
    torch.hub.download_url_to_file(pretrained_url, pretrained_path)
    print(f"Downloaded pretrained weights to {pretrained_path}")

# Load the model with the correct input size and number of classes
model = mamba_vision_L(pretrained=False, resolution=512, num_classes=12)

# Replace the head with the correct number of output classes
num_features = model.head.in_features
model.head = torch.nn.Linear(num_features, 12)

# Load the pretrained weights, ignoring mismatched weights
checkpoint = torch.load(pretrained_path, map_location='cpu')
state_dict = checkpoint['state_dict'] if 'state_dict' in checkpoint else checkpoint

# Filter out the `head` layer weights
filtered_state_dict = {k: v for k, v in state_dict.items() if not k.startswith('head')}
model.load_state_dict(filtered_state_dict, strict=False)

print("Model loaded successfully with modified head for 12 classes.")
model = model.to(device)


/scratch/sahil/anaconda3/envs/dhs/lib/python3.11/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/scratch/sahil/anaconda3/envs/dhs/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Model loaded successfully with modified head for 12 classes.


#### ConvNeXt

In [ ]:
from torchvision.models import convnext_base, ConvNeXt_Base_Weights

class AugmentHead(nn.Module):
    def __init__(self, dim:int):
        super().__init__()
        self.augment_head = nn.Linear(dim, 12)
        
    def forward(self, x):
        x = self.augment_head(x)
        return x

class ConvNeXtB(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = convnext_base(weights=ConvNeXt_Base_Weights.DEFAULT)
        self.backbone.classifier[2] = AugmentHead(dim=self.backbone.classifier[2].in_features)
        
    def forward(self, x):
        return self.backbone(x)



model = ConvNeXtB()
checkpoint_path = '/scratch/genai/dog_heart/models/custom2/20250504_073224/models/bm_255.pth'

model.load_state_dict(torch.load(checkpoint_path, map_location='cpu',weights_only=True))
model = model.to(device)

#### Efficient Net 7B

In [17]:
class AugmentHead(nn.Module):
    def __init__(self, dim:int):
        super().__init__()
        self.augment_head = nn.Linear(dim, 12)
        
    def forward(self, x):
        x = self.augment_head(x)
        return x

class EfficientNetB7(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = models.efficientnet_b7(weights='DEFAULT')
        self.backbone.classifier[1] = AugmentHead(dim=self.backbone.classifier[1].in_features)

    def forward(self, x):
        return self.backbone(x)


checkpoint_path="/scratch/genai/dog_heart/models/efficient_net-7b/20250501_040140/models/bm_592.pth"
model = EfficientNetB7()

model.load_state_dict(torch.load(checkpoint_path, map_location='cpu',weights_only=True))
model = model.to(device)
        

In [16]:
#test_accuracy(model, test_loader, device, calc_vhs, get_labels)


In [ ]:
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import torch
import numpy as np

PREDICT_FOLDER = "predictions_effNet" # Change the folder according to the selected model. 
os.makedirs(PREDICT_FOLDER, exist_ok=True)

In [15]:
IMAGE_FOLDER = "models/data/Test_Images/Images"
img_size = 512
total_images = len(test_loader.dataset)
processed_images = 0

model.eval()

def save_predictions_to_mat(image_name, predicted_points, vhs_value):
    """Save predicted points and VHS in MATLAB-compatible .mat format."""
    mat_filename = os.path.splitext(image_name)[0] + ".mat"
    mat_filepath = os.path.join(PREDICT_FOLDER, mat_filename)

    # Convert points to correct MATLAB format
    mat_data = {
        "six_points": np.array(predicted_points),  # Store 6 points in an array
        "VHS": np.array([[vhs_value]])  # Store VHS as a 2D array
    }

    sio.savemat(mat_filepath, mat_data)
    print(f"✅ Predictions saved in MATLAB format: {mat_filepath}")


def calculateVHS(A,B,C,D,E,F):
    # Calculate distances using Euclidean formula
    AB = np.linalg.norm(B - A)  
    CD = np.linalg.norm(D - C)  
    EF = np.linalg.norm(F - E) 

    # Calculate VHS
    VHS = 6 * (AB + CD) / EF
    return VHS
    
with torch.no_grad():
    for images, names in test_loader:
        images = images.to(device)
        outputs = model(images)
        outputs = outputs.cpu().numpy()
        outputs = outputs.reshape(outputs.shape[0], 6, 2)

        for i, points in enumerate(outputs):
            img = Image.open(f'{IMAGE_FOLDER}/{names[i]}')
            
            # Get original image size and return predicted points back to original points size
            w, h = img.size
            points = points.reshape(-1, 2)
            points = points * img_size
            
            points[:, 0] = w / img_size * points[:, 0]
            points[:, 1] = h / img_size * points[:, 1]
            
            vhs = calculateVHS(points[0], points[1], points[2], points[3], points[4], points[5])

            save_predictions_to_mat(names[i], points, vhs)

            processed_images += 1
            progress = round((processed_images * 100) / total_images, 0)
            

✅ Predictions saved in MATLAB format: predictions_convnext/11037.mat
✅ Predictions saved in MATLAB format: predictions_convnext/11037_3.mat
✅ Predictions saved in MATLAB format: predictions_convnext/11038.mat
✅ Predictions saved in MATLAB format: predictions_convnext/11038_3.mat
✅ Predictions saved in MATLAB format: predictions_convnext/11039.mat
✅ Predictions saved in MATLAB format: predictions_convnext/11040.mat
✅ Predictions saved in MATLAB format: predictions_convnext/11040_3.mat
✅ Predictions saved in MATLAB format: predictions_convnext/11041.mat
✅ Predictions saved in MATLAB format: predictions_convnext/11041_3.mat
✅ Predictions saved in MATLAB format: predictions_convnext/11042.mat
✅ Predictions saved in MATLAB format: predictions_convnext/11042_3.mat
✅ Predictions saved in MATLAB format: predictions_convnext/11043.mat
✅ Predictions saved in MATLAB format: predictions_convnext/11044.mat
✅ Predictions saved in MATLAB format: predictions_convnext/11045.mat
✅ Predictions saved in M